In [59]:
import pandas as pd
import spacy
from spacy.matcher import PhraseMatcher
import collections as c

nlp = spacy.load("en_core_web_lg")

all_stopwords = nlp.Defaults.stop_words

In [7]:
df = pd.read_csv('book_1.csv')

df.head()

,chapter_number,chapter_title,tokens,pos,lemma,tag,dep,sentence,word,entity,title
0,Chapter 1,THE NEW POSTMASTER,wrong,ADJ,wrong,JJ,acomp,\n I’ve been wrong before.,NaN,NaN,NaN
1,Chapter 1,THE NEW POSTMASTER,,SPACE,,_SP,ROOT,,NaN,NaN,NaN
2,Chapter 1,THE NEW POSTMASTER,Oh,INTJ,oh,UH,intj,"Oh heck, if I’m being real honest, I’ve been w...",NaN,NaN,NaN
3,Chapter 1,THE NEW POSTMASTER,heck,INTJ,heck,UH,intj,"Oh heck, if I’m being real honest, I’ve been w...",NaN,NaN,NaN
4,Chapter 1,THE NEW POSTMASTER,real,ADV,real,RB,advmod,"Oh heck, if I’m being real honest, I’ve been w...",NaN,NaN,NaN


In [71]:
C_num = 1
C_num = str(C_num)

#chapter_df = df[df['chapter_number'] ==  'Chapter '+ C_num]
#persons_df = chapter_df[chapter_df['entitiy'] == 'PERSON']
persons_df = df[df['entity'] == 'PERSON']
persons_count_df = persons_df.groupby(by = ['word']).count().reset_index()
persons_count_df = persons_count_df.drop(['chapter_number','chapter_title','tokens','lemma','tag','dep','pos','title','sentence'], axis = 1)


In [72]:
persons_count_df.sort_values(by = 'entity', ascending = False).head(40)

,word,entity
14,Emma,740
55,Walker,190
11,Elbert,142
9,Dit,138
18,Griffith,131
46,Seay,116
40,Pearl,95
4,Bobby,95
19,Haley,89
41,Pooley,73


In [25]:
 # convert list to string 
def listToString(s): 
    str1 = ""  
    for item in s: 
        str1 += item
        str1 += ' '
    return str1 

In [28]:
persons_sentence = persons_df['sentence'].unique().tolist()
persons_string = listToString(persons_sentence)

persons_word = persons_df['word'].unique().tolist()
terms = []
for persons in persons_word:
    terms.append("{} said".format(persons))
    terms.append("said {}".format(persons))

In [56]:
matcher = PhraseMatcher(nlp.vocab)

patterns = [nlp.make_doc(text) for text in terms]
matcher.add("TerminologyList", patterns)

doc = nlp(persons_string)

matches = matcher(doc)
spoken_persons = []
for match_id, start, end in matches:
    span = str(doc[start:end])
    span2 = span.replace(' said', '')
    span3 = span2.replace('said ', '')
    spoken_persons.append(span3)

In [67]:
spoken_dict = c.Counter(spoken_persons)
spoken_df = pd.DataFrame.from_dict(spoken_dict, orient='index', columns = ['Count']).reset_index()
spoken_df.sort_values(by = 'Count', ascending = False)

,index,Count
1,Emma,72
4,Elbert,13
9,Bobby,11
10,Seay,11
2,Walker,8
8,Pearl,7
13,Griffith,7
5,Raymond,6
15,Haley,2
3,Ulman,2


In [76]:
spoken_hurdle = spoken_df[spoken_df['Count'] > 3]
spoken_hurdle = spoken_hurdle['index'].tolist()
mention_hurdle = persons_count_df[persons_count_df['entity'] > 10]
mention_hurdle = mention_hurdle['word'].tolist()

In [77]:
def check_person (person_list):
    book_characters = []
    for person in person_list:
        if person in spoken_hurdle:
            book_characters.append(person)
        elif person in mention_hurdle:
            book_characters.append(person)
            
    return book_characters

In [81]:
persons_list1 = persons_df['word'].unique().tolist()

In [82]:
book1_characters = check_person (persons_list1)

book1_characters

['Emma',
 'Walker',
 'Sims',
 'Dit',
 'Ollie',
 'Ulman',
 'Elman',
 'Raymond',
 'Earl',
 'Pearl',
 'Robert',
 'Lois',
 'Pooley',
 'Wiggens',
 'Elbert',
 'Davidson',
 'Haley',
 'Griffith',
 'Jim',
 'Dang',
 'Mitch',
 'Seay',
 'Bobby',
 'Mary']

In [ ]:
# then on chapter you have a persons list and you compare to the book character list
# if it is in the list then that is a character for the chapter